In [ ]:
import os
import random
import shutil

#Paths
dataset_path = "Data/YOLO"
output_dir = "Data/split_dataset"
yaml_file = os.path.join(output_dir, "mydataset.yml")

#Säkerställer att mappar finns
for subset in ["train", "val", "test"]:
    os.makedirs(os.path.join(output_dir, subset, "images"), exist_ok=True)
    os.makedirs(os.path.join(output_dir, subset, "labels"), exist_ok=True)

#Split ratio
train_ratio = 0.80
val_ratio = 0.10
test_ratio = 0.10

#hämtar alla bild filer (stöder jpg, png, jpeg)
image_files = [f for f in os.listdir(os.path.join(dataset_path, "images")) if f.endswith((".jpg", ".png", ".jpeg"))]
random.shuffle(image_files)

#Delar datasettet
train_count = int(len(image_files) * train_ratio)
val_count = int(len(image_files) * val_ratio)

train_images = image_files[:train_count]
val_images = image_files[train_count:train_count + val_count]
test_images = image_files[train_count + val_count:]

#Flyttar bilder och labels
def move_files(image_list, subset_name):
    for img in image_list:
        img_path = os.path.join(dataset_path, "images", img)
        label_path = os.path.join(dataset_path, "labels", os.path.splitext(img)[0] + ".txt")

        #Flyttar bilden
        shutil.move(img_path, os.path.join(output_dir, subset_name, "images", img))

        #Flyttar annotation fil om den finns
        if os.path.exists(label_path):
            shutil.move(label_path, os.path.join(output_dir, subset_name, "labels", os.path.splitext(img)[0] + ".txt"))

#flyttar filer i respektive mappar
move_files(train_images, "train")
move_files(val_images, "val")
move_files(test_images, "test")

#Flyttar classes.txt om sådan finns
classes_file = os.path.join(dataset_path, "classes.txt")
if os.path.exists(classes_file):
    shutil.copy(classes_file, output_dir)

#Hämtar classnamn från classes.txt
def get_class_names():
    if os.path.exists(classes_file):
        with open(classes_file, "r") as f:
            return [line.strip() for line in f.readlines()]
    return []

class_names = get_class_names()    

#Genererar `mydataset.yml` fil
with open(yaml_file, "w") as f:
    f.write(f"# mydataset.yml - YOLO dataset configuration\n\n")
    f.write(f"path:   #Root directory av datasettet OBS MÅSTE VARA FULL PATH (C:\...)\n")
    f.write(f"train: train/images  #Tränings mappen\n")
    f.write(f"val: val/images  #Validation mappen\n")
    f.write(f"test: test/images  #Test mappen\n\n")
    f.write(f"#Antal klasser\n")
    f.write(f"nc: {len(class_names)}\n\n")
    f.write(f"#Klass namn\n")
    f.write("names:\n")
    for i, name in enumerate(class_names):
        f.write(f"  {i}: {name}\n")

print(f"Dataset split completed! Train: {len(train_images)}, Val: {len(val_images)}, Test: {len(test_images)}")
print(f"`mydataset.yml` created with {len(class_names)} classes.")

Dataset split completed! Train: 0, Val: 0, Test: 0
`mydataset.yml` created with 2 classes.
